# **Projeto de Previsão de Inadimplência**

---
## **Motivação:**
A inadimplência é um problema crítico para instituições financeiras, uma vez que impacta diretamente o fluxo de caixa e a saúde financeira das empresas. Prever com antecedência quais clientes têm maior chance de se tornarem inadimplentes permite que as empresas adotem estratégias preventivas, minimizando prejuízos.



---
## **Objetivos:**

Este projeto tem como **principal objetivo** construir um modelo de Machine Learning que seja capaz de prever a inadimplência de clientes com base em um conjunto de dados de contratos de financiamento.

- **Construir um modelo preditivo de inadimplência** utilizando algoritmos de classificação.
- **Analisar as variáveis mais relevantes** para prever a inadimplência, ajudando a entender os fatores de risco.
- **Avaliar a performance dos modelos** utilizando métricas de performance.


---

#### Entre as colunas deste dataset temos:
| Coluna                        | Descrição                                          |
|-------------------------------|----------------------------------------------------|
| **NUMERO_CONTRATO**            | Número identificador do contrato                   |
| **DATA_ASSINATURA_CONTRATO**   | Data em que o contrato foi assinado                |
| **TIPO_FINANCIAMENTO**         | Tipo de financiamento oferecido                    |
| **TAXA_AO_ANO**                | Taxa de juros anual do contrato                    |
| **PZ_FINANCIAMENTO**           | Prazo de financiamento (em meses)                  |
| **CIDADE_CLIENTE**             | Cidade onde o cliente reside                       |
| **ESTADO_CLIENTE**             | Estado onde o cliente reside                       |
| **RENDA_MENSAL_CLIENTE**       | Renda mensal do cliente                            |
| **QT_PC_ATRASO**               | Quantidade de parcelas em atraso                   |
| **QT_DIAS_PRIM_PC_ATRASO**     | Quantidade de dias em atraso da primeira parcela   |
| **QT_TOTAL_PC_PAGAS**          | Quantidade total de parcelas pagas                 |
| **VL_TOTAL_PC_PAGAS**          | Valor total das parcelas pagas                     |
| **QT_PC_PAGAS_EM_DIA**         | Quantidade de parcelas pagas em dia                |
| **QT_DIAS_MIN_ATRASO**         | Menor quantidade de dias de atraso                 |
| **QT_DIAS_MAX_ATRASO**         | Maior quantidade de dias de atraso                 |
| **QT_DIAS_MEDIA_ATRASO**       | Média de dias em atraso                            |
| **VALOR_FINANCIAMENTO**        | Valor total do financiamento                       |
| **VALOR_PARCELA**              | Valor de cada parcela                              |
| **IDADE_DATA_ASSINATURA_CONTRATO** | Idade do cliente no momento da assinatura do contrato |
| **INADIMPLENTE_COBRANCA**      | Indicador se o cliente está inadimplente           |



**A coluna alvo é "INADIMPLENTE_COBRANCA"**

#### Este é um problema de **Classificação**

---

## **Etapas do projeto**

1. **Carregamento e análise inicial dos dados**: Vamos carregar o dataset e verificar suas características iniciais.

2. **Exploração e tratamento dos dados**: Identificar dados ausentes, remover outliers (se necessário), e preparar as variáveis para modelagem.

3. **Análise Estatística e Engenharia de Atributos**: Para entender as características e padrões dos dados e melhorar a qualidade dos dados.

4. **Visualização com Gráficos**: Visualizar os dados de uma forma gráfica para ajudar na compreensão.

5. **Treinamento do modelo**: Construir e treinar modelos preditivos usando algoritmos de Machine Learning.

6. **Avaliação do modelo**: Avaliar a performance dos modelos com base nas métricas adequadas.

7. **Colocar o modelo em producão**: Para que as previsões e insights gerados pelo modelo possam ser utilizados em cenários do mundo real.

8. **Conclusão**: Resumo dos resultados e insights finais.

---

### Conectando com o Banco de dados e Importando os dados

In [7803]:
#importando as bibliotecas
import warnings
import pandas as pd
import pypyodbc as sql
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from dotenv import load_dotenv

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
warnings.filterwarnings("ignore")

In [7804]:
#carregando as variaveis de ambiente
load_dotenv()

USUARIO = os.environ.get("USUARIO")
SENHA = os.environ.get("SENHA")
HOST = os.environ.get("HOST")
DATABASE = os.environ.get("DATABASE")


In [7805]:
#importar os dados do SQL

connection_string = "Driver={SQL Server};Server=" + HOST + ";Database=" + DATABASE + ";UID=" + USUARIO + ";PWD=" + SENHA + ";"

conexao = sql.connect(connection_string)

df_original = pd.read_sql_query("SELECT * FROM EXTRACAO_DADOS_SISTEMA", conexao)

conexao.close()

### Começando a Análise Exploratória

In [7806]:
#observando o numero de linhas e colunas
print(f"O dataset contém {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.")

O dataset contém 10415 linhas e 20 colunas.


In [7807]:
#observando todas as colunas e os tipos das mesmas, depois, iremos otimizar este dataframe
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   numero_contrato                 10415 non-null  int64  
 1   data_assinatura_contrato        10415 non-null  object 
 2   tipo_financiamento              10415 non-null  object 
 3   taxa_ao_ano                     10415 non-null  float64
 4   pz_financiamento                10415 non-null  int64  
 5   cidade_cliente                  10415 non-null  object 
 6   estado_cliente                  10415 non-null  object 
 7   renda_mensal_cliente            10415 non-null  float64
 8   qt_pc_atraso                    10415 non-null  int64  
 9   qt_dias_prim_pc_atraso          10415 non-null  int64  
 10  qt_total_pc_pagas               10415 non-null  int64  
 11  vl_total_pc_pagas               10411 non-null  float64
 12  qt_pc_paga_em_dia               

In [7808]:
#visao geral dos dados
df_original.sample(10)

,numero_contrato,data_assinatura_contrato,tipo_financiamento,taxa_ao_ano,pz_financiamento,cidade_cliente,estado_cliente,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato,inadimplente_cobranca
9082,84976,2018-02-23,IMOBILIARIO,20.00,72,SAO PAULO,SP,9800.00,0,0,28,23957.52,28,0,0,0,420000.00,6000.00,29.00,SIM
2658,50162,2018-05-11,IMOBILIARIO,16.00,180,BRASILIA,DF,5800.00,38,1151,16,12354.35,13,5,37,16,119208.00,768.23,38.00,SIM
1337,79769,2017-02-21,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,36,24963.74,15,18,112,63,350000.00,4979.17,36.00,NAO
10315,139185,2019-08-21,IMOBILIARIO,16.50,72,GOIANIA,GO,1800.00,37,1106,1,788.99,1,0,0,0,231104.16,3285.44,56.00,NAO
5717,76870,2014-02-14,IMOBILIARIO,17.00,72,CUIABA,MT,7800.00,65,3066,4,2726.84,3,1,1,1,280000.00,3983.33,32.00,NAO
3935,153499,2022-07-06,IMOBILIARIO,14.80,100,ITABERABA,BA,1800.00,0,0,4,2273.76,2,4,4,4,420000.00,4288.80,26.00,SIM
1376,138525,2019-07-05,IMOBILIARIO,16.40,240,SAO PAULO,SP,1800.00,0,0,41,77983.59,31,2,49,11,500498.49,2427.42,36.00,SIM
1295,76549,2013-06-25,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,64,40688.76,55,2,17,6,210000.00,2983.33,46.00,SIM
9770,80607,2017-08-25,IMOBILIARIO,19.00,72,AGUAS LINDAS DE GOIAS,GO,9800.00,0,0,30,13183.32,30,0,0,0,210000.00,2995.83,46.00,SIM
3206,132516,2017-10-24,IMOBILIARIO,20.00,100,SAO PAULO,SP,1800.00,0,0,49,54570.03,47,1,1,1,490000.00,5040.00,65.00,NAO


In [7809]:
#vendo metricas das colunas numericas
df_original.describe()

,numero_contrato,taxa_ao_ano,pz_financiamento,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato
count,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10411.00,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10414.00
mean,95569.36,17.27,114.24,5620.26,16.72,664.39,35.95,44528.55,27.67,6.12,79.13,28.84,325590.86,3617.34,40.79
std,34561.66,2.57,64.33,2930.30,30.71,1181.35,28.38,65640.97,25.15,82.91,334.87,137.40,177477.49,1932.71,12.39
min,32709.00,7.00,48.00,1800.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,29327.00,185.74,0.00
25%,75868.50,16.00,72.00,1800.00,0.00,0.00,11.00,8675.44,7.00,0.00,0.00,0.00,210000.00,2467.76,31.25
50%,79111.00,17.00,72.00,7800.00,0.00,0.00,33.00,24345.60,22.00,1.00,8.00,4.00,280000.00,3470.59,39.00
75%,136637.50,19.00,180.00,7800.00,22.00,1005.50,54.00,46787.29,42.00,3.00,62.00,22.00,392973.77,4930.25,49.00
max,155890.00,25.50,240.00,9800.00,178.00,5655.00,167.00,714499.74,136.00,5465.00,5710.00,5480.00,1400000.00,14410.00,118.00


- Aqui observamos alguns potenciais outliers como idade_data_assinatura_contrato minima: 0 e idade_data_assinatura_contrato maxima: 118
---

In [7810]:
#vendo metricas das colunas nao numericas
df_original.describe(exclude="number")

,data_assinatura_contrato,tipo_financiamento,cidade_cliente,estado_cliente,inadimplente_cobranca
count,10415,10415,10415,10415,10415
unique,1070,1,525,27,2
top,2014-06-13,IMOBILIARIO,BRASILIA,DF,SIM
freq,187,10415,3737,4240,8038


In [7811]:
#filtro para ver a quantidade media de parcelas em atraso por renda mensal do cliente
filtro_atraso = df_original.groupby("renda_mensal_cliente")["qt_pc_atraso"].mean()
print(filtro_atraso)

maior_id_atraso = filtro_atraso.idxmax()
maior_media_atraso = filtro_atraso.max()

print(f"\nA renda_mensal_cliente com a maior média de qt_pc_atraso é R$ {maior_id_atraso} com uma média de {maior_media_atraso:.2f} de parcelas em atraso")

renda_mensal_cliente
1800.00    5.60
3800.00    0.06
4800.00    8.12
5800.00   31.69
6800.00   26.58
7800.00   22.82
9800.00   16.73
Name: qt_pc_atraso, dtype: float64

A renda_mensal_cliente com a maior média de qt_pc_atraso é R$ 5800.0 com uma média de 31.69 de parcelas em atraso


- A quantidade média de parcelas atrasadas é bem maior quando o cliente tem uma renda mensal de mais de **R$ 5.800.**

---

In [7812]:
#filtro para ver a idade media do cliente na data de assinatura do contrato por renda mensal do cliente
df_original.groupby("renda_mensal_cliente")["idade_data_assinatura_contrato"].mean()

renda_mensal_cliente
1800.00   40.98
3800.00   45.00
4800.00   42.25
5800.00   41.90
6800.00   44.77
7800.00   40.37
9800.00   39.48
Name: idade_data_assinatura_contrato, dtype: float64

- A média de idade na hora da assinatura do contrato é bem parecida em todos os níveis de renda com a média total.

---

In [7813]:
#filtro para ver a quantidade media do valor das parcelas por renda mensal do cliente
filtro_renda = df_original.groupby("renda_mensal_cliente")["valor_parcela"].mean()
print(filtro_renda)

maior_id_venda = filtro_renda.idxmax()
maior_media_venda = filtro_renda.max()

print(f"\nA renda_mensal_cliente com a maior média de valor_parcela é R$ {maior_id_venda} com uma média de R$ {maior_media_venda:.2f} por parcela")



renda_mensal_cliente
1800.00   4733.81
3800.00   1143.67
4800.00   1040.42
5800.00   1265.53
6800.00   2843.32
7800.00   3498.73
9800.00   3910.99
Name: valor_parcela, dtype: float64

A renda_mensal_cliente com a maior média de valor_parcela é R$ 1800.0 com uma média de R$ 4733.81 por parcela


- A média do valor da parcela por renda mensal do cliente é bem maior quando o cliente recebe **R$ 1.800 ou menos**, ou, quando o cliente recebe **R$ 6.800 ou mais**.

---

In [7814]:
#observando o inicio e fim dos dados da coluna data_assinatura_contrato
inicio = pd.to_datetime(df_original["data_assinatura_contrato"]).dt.date.min() 
fim = pd.to_datetime(df_original["data_assinatura_contrato"]).dt.date.max()

print("Período dos dados de:", inicio, "até", fim)

Período dos dados de: 2012-01-10 até 2022-11-14


- Aqui observamos que só existe um tipo de financiamento nestes dados: **IMOBILIARIO**
- Clientes de **525 cidades diferentes**
- Clientes de **todos** os estados do Brasil
- Maior parte dos clientes neste conjunto de dados está **inadimplente (8038 de 10415, ~77%)**
- Período dos dados de: **2012-01-10 até 2022-11-14**


---

In [7815]:
#filtro para ver a porcentagem de valores por renda mensal do cliente
print(df_original["renda_mensal_cliente"].value_counts(normalize=True))
print(f"\nA maior parte dos clientes ({df_original["renda_mensal_cliente"].value_counts().max()}), recebe entre R$ 6.900 e R$ {df_original["renda_mensal_cliente"].value_counts().idxmax()}")

renda_mensal_cliente
7800.00   0.41
1800.00   0.34
5800.00   0.10
9800.00   0.09
4800.00   0.04
6800.00   0.01
3800.00   0.00
Name: proportion, dtype: float64

A maior parte dos clientes (4306), recebe entre R$ 6.900 e R$ 7800.0


- A maior parte dos clientes (**41%**) recebe entre **R$ 6.900 e R$ 7.800 mensais**
- **34%** dos clientes recebem até **R$ 1.800**
- **75%** dos cliente ou recebem entre **R$ 6.900 e R$ 7.800 ou até R$ 1.800**
          
---

In [7816]:
#visualizando a quantidade de linhas onde a idade na data de assinatura é maior ou menor que um valor
total = len(df_original.query("idade_data_assinatura_contrato < 10"))
total2 = len(df_original.query("idade_data_assinatura_contrato < 20"))
total3 = len(df_original.query("idade_data_assinatura_contrato > 50"))
total4 = len(df_original.query("idade_data_assinatura_contrato > 70"))
print(f"Número de linhas onde a idade_data_assinatura_contrato é menor que 10: {total}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é menor que 20: {total2}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é maior que 50: {total3}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é maior que 70: {total4}")


Número de linhas onde a idade_data_assinatura_contrato é menor que 10: 7
Número de linhas onde a idade_data_assinatura_contrato é menor que 20: 80
Número de linhas onde a idade_data_assinatura_contrato é maior que 50: 2303
Número de linhas onde a idade_data_assinatura_contrato é maior que 70: 175


- Existem **7** registros onde a idade na data de assinatura é menor que 10 anos, um possível erro 
- **80** clientes com menos de **20 anos** na data de assinatura
- **2303** clientes com mais de **50 anos** na data de assinatura
- **175** clientes com mais de **70 anos** na data de assinatura
          
---

In [7817]:
#porcentagem de parcelas pagas em dia e em atraso

total_pagas = df_original["qt_total_pc_pagas"].sum()
pagas_em_dia = df_original["qt_pc_paga_em_dia"].sum()

porcentagem_em_dia = (pagas_em_dia / total_pagas) * 100
porcentagem_atraso = 100 - porcentagem_em_dia

print(f"Porcentagem de parcelas pagas em dia: {porcentagem_em_dia:.2f}%")
print(f"Porcentagem de parcelas pagas em atraso: {porcentagem_atraso:.2f}%")


Porcentagem de parcelas pagas em dia: 76.97%
Porcentagem de parcelas pagas em atraso: 23.03%


- **76%** das parcelas pagas, são pagas em dia
---

In [7818]:
#verificando a quantidade de valores únicos de cada coluna
df_original.nunique()

numero_contrato                   10415
data_assinatura_contrato           1070
tipo_financiamento                    1
taxa_ao_ano                         111
pz_financiamento                      9
cidade_cliente                      525
estado_cliente                       27
renda_mensal_cliente                  7
qt_pc_atraso                        130
qt_dias_prim_pc_atraso              227
qt_total_pc_pagas                   137
vl_total_pc_pagas                  9860
qt_pc_paga_em_dia                   129
qt_dias_min_atraso                  122
qt_dias_max_atraso                  534
qt_dias_media_atraso                323
valor_financiamento                 834
valor_parcela                      1826
idade_data_assinatura_contrato       78
inadimplente_cobranca                 2
dtype: int64

- Verificamos novamente que **tipo_financiamento** possuí apenas um tipo de dado
- **vl_total_pc_pagas** possuí alta cardinalidade 
- Apenas 7 tipos diferentes de **renda_mensal_cliente**                
---

Faremos mais análises na parte de **Visualização com gráficos**


### Otimização do dataframe

In [7819]:
#otimizando a coluna Data_Contratacao para datetime64[ns]                  
df_original["data_assinatura_contrato"] = pd.to_datetime(df_original["data_assinatura_contrato"])

In [7820]:
#otimizando colunas categoricas
colunas_categoricas = ["tipo_financiamento", "cidade_cliente", "estado_cliente", "inadimplente_cobranca"]

for coluna in colunas_categoricas:
	df_original[coluna] = df_original[coluna].astype("category")


In [7821]:
#otimizando colunas numericas
colunas_float = df_original.select_dtypes(include="float64").columns
colunas_int = df_original.select_dtypes(include="int64").columns

df_original[colunas_float] = df_original[colunas_float].apply(pd.to_numeric, downcast="float")
df_original[colunas_int] = df_original[colunas_int].apply(pd.to_numeric, downcast="integer")


In [7822]:
#verificando a mudança de tamanho do dataset
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   numero_contrato                 10415 non-null  int32         
 1   data_assinatura_contrato        10415 non-null  datetime64[ns]
 2   tipo_financiamento              10415 non-null  category      
 3   taxa_ao_ano                     10415 non-null  float32       
 4   pz_financiamento                10415 non-null  int16         
 5   cidade_cliente                  10415 non-null  category      
 6   estado_cliente                  10415 non-null  category      
 7   renda_mensal_cliente            10415 non-null  float32       
 8   qt_pc_atraso                    10415 non-null  int16         
 9   qt_dias_prim_pc_atraso          10415 non-null  int16         
 10  qt_total_pc_pagas               10415 non-null  int16         
 11  vl

- O tamanho do dataframe diminuiu de ***1.6+ MB para 683.0 KB**
---

### Tratamento dos dados

In [7823]:
#removendo linhas duplicadas 
df_original.drop_duplicates()
print(f"O dataset contém {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.")

O dataset contém 10415 linhas e 20 colunas.


- Não foram encontradas linhas duplicadas
---

In [7824]:
#verificando valores nulos
print(f"O número de valores vazios de cada coluna \n\n{df_original.isnull().sum()}")

O número de valores vazios de cada coluna 

numero_contrato                   0
data_assinatura_contrato          0
tipo_financiamento                0
taxa_ao_ano                       0
pz_financiamento                  0
cidade_cliente                    0
estado_cliente                    0
renda_mensal_cliente              0
qt_pc_atraso                      0
qt_dias_prim_pc_atraso            0
qt_total_pc_pagas                 0
vl_total_pc_pagas                 4
qt_pc_paga_em_dia                 0
qt_dias_min_atraso                0
qt_dias_max_atraso                0
qt_dias_media_atraso              0
valor_financiamento               0
valor_parcela                     0
idade_data_assinatura_contrato    1
inadimplente_cobranca             0
dtype: int64


- Identificamos que ***vl_total_pc_pagas**  e **idade_data_assinatura_contrato** possuem 4 e 1 valor vazio respectivamente, iremos excluir essas linhas já que são poucas e não irão afetar o modelo
---

In [7825]:
#excluindo linhas com valores nulos
df_original.dropna(inplace=True)

In [7826]:
#verificando novamente se existem valores nulos
print(f"O número de valores vazios de cada coluna \n\n{df_original.isnull().sum()}")


O número de valores vazios de cada coluna 

numero_contrato                   0
data_assinatura_contrato          0
tipo_financiamento                0
taxa_ao_ano                       0
pz_financiamento                  0
cidade_cliente                    0
estado_cliente                    0
renda_mensal_cliente              0
qt_pc_atraso                      0
qt_dias_prim_pc_atraso            0
qt_total_pc_pagas                 0
vl_total_pc_pagas                 0
qt_pc_paga_em_dia                 0
qt_dias_min_atraso                0
qt_dias_max_atraso                0
qt_dias_media_atraso              0
valor_financiamento               0
valor_parcela                     0
idade_data_assinatura_contrato    0
inadimplente_cobranca             0
dtype: int64


- Todos os valores nulos foram excluídos
---

In [7827]:
#verificando a possibilidade de outliers
df_original[df_original["idade_data_assinatura_contrato"] <= 17]

,numero_contrato,data_assinatura_contrato,tipo_financiamento,taxa_ao_ano,pz_financiamento,cidade_cliente,estado_cliente,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato,inadimplente_cobranca
856,79171,2017-11-14,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,35,35294.51,10,1,55,30,412755.00,5871.93,9.00,SIM
1418,79545,2016-11-11,IMOBILIARIO,20.00,72,BRASILIA,DF,7800.00,0,0,38,33545.05,3,1,245,61,420000.00,6000.00,8.00,SIM
2044,137914,2019-04-09,IMOBILIARIO,14.65,72,PLANALTINA DF,DF,1800.00,0,0,44,31670.84,42,2,2,2,385000.00,5459.13,12.00,SIM
2219,79450,2016-04-12,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,53,22320.79,41,1,82,21,210000.00,2987.50,17.00,SIM
2253,76537,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,56,28877.66,44,11,178,61,175000.00,2486.11,16.00,SIM
2979,76481,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,63,35871.16,44,8,68,34,175000.00,2486.11,16.00,SIM
3002,136455,2018-12-11,IMOBILIARIO,14.65,72,SAO PAULO,SP,1800.00,0,0,43,27952.17,37,2,21,5,280000.00,3970.28,0.00,SIM
3142,75101,2012-01-10,IMOBILIARIO,15.00,72,PLANALTINA,GO,7800.00,59,3769,6,5106.58,4,20,49,34,373590.00,5299.94,17.00,NAO
3562,136456,2018-12-11,IMOBILIARIO,14.65,72,SAO PAULO,SP,1800.00,0,0,43,27955.99,37,2,21,8,280000.00,3970.28,0.00,SIM
3749,78832,2016-02-16,IMOBILIARIO,17.00,72,ALTAMIRA,PA,7800.00,0,0,51,34921.82,11,1,154,43,210000.00,2987.50,7.00,SIM


- Alguns destes valores aparentam ser outliers, mas como são poucos, eles serão tratados na parte de Normalização/ Padronização dos dados
---

In [7828]:
#eliminando colunas inúteis para a nossa análise

df_original = df_original.drop(["numero_contrato", "data_assinatura_contrato", "tipo_financiamento"], axis=1)


- Eliminando **numero_contrato** pois não tem relevância para nossa análise
- Eliminando **data_assinatura_contrato** pois não se trata de uma série temporal
- Eliminando **tipo_financiamento** pois todas as linhas possuem o mesmo valor

---

In [7829]:
#transformando nossas colunas para caixa alta
df_original.columns = df_original.columns.str.upper()

In [7830]:
#renomeando algumas colunas

df_original.rename(columns={"VALOR_FINANCIAMENTO": "VL_FINANCIAMENTO", "VALOR_PARCELA": "VL_PARCELA",  }, inplace=True)

In [7831]:
#visualizando novamente as colunas

df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   TAXA_AO_ANO                     10410 non-null  float32 
 1   PZ_FINANCIAMENTO                10410 non-null  int16   
 2   CIDADE_CLIENTE                  10410 non-null  category
 3   ESTADO_CLIENTE                  10410 non-null  category
 4   RENDA_MENSAL_CLIENTE            10410 non-null  float32 
 5   QT_PC_ATRASO                    10410 non-null  int16   
 6   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int16   
 7   QT_TOTAL_PC_PAGAS               10410 non-null  int16   
 8   VL_TOTAL_PC_PAGAS               10410 non-null  float64 
 9   QT_PC_PAGA_EM_DIA               10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO              10410 non-null  int16   
 11  QT_DIAS_MAX_ATRASO              10410 non-null  int16   
 12  QT_DIAS_MEDIA_ATRASO   

## Engenharia de Atributos

In [7832]:
#verificando a cardinalidade das colunas
df_original.nunique()

TAXA_AO_ANO                        111
PZ_FINANCIAMENTO                     9
CIDADE_CLIENTE                     525
ESTADO_CLIENTE                      27
RENDA_MENSAL_CLIENTE                 7
QT_PC_ATRASO                       130
QT_DIAS_PRIM_PC_ATRASO             227
QT_TOTAL_PC_PAGAS                  136
VL_TOTAL_PC_PAGAS                 9859
QT_PC_PAGA_EM_DIA                  129
QT_DIAS_MIN_ATRASO                 122
QT_DIAS_MAX_ATRASO                 534
QT_DIAS_MEDIA_ATRASO               323
VL_FINANCIAMENTO                   834
VL_PARCELA                        1826
IDADE_DATA_ASSINATURA_CONTRATO      78
INADIMPLENTE_COBRANCA                2
dtype: int64

In [7833]:
#verificando valores minimos e maximos para algumas colunas 

df_original[["VL_TOTAL_PC_PAGAS", "VL_FINANCIAMENTO", "VL_PARCELA", "IDADE_DATA_ASSINATURA_CONTRATO"]].describe().loc[["min", "mean", "max", "25%", "50%", "75%"]]


,VL_TOTAL_PC_PAGAS,VL_FINANCIAMENTO,VL_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO
min,0.01,29327.00,185.74,0.00
mean,44530.61,325570.97,3617.72,40.79
max,714499.74,1400000.00,14410.00,118.00
25%,8674.22,210000.00,2471.04,31.25
50%,24345.61,280000.00,3470.59,39.00
75%,46807.04,392973.77,4931.86,49.00


In [7834]:
#criando faixa de valores

faixas = [-100, 8_675, 25_000, 47_000, 9_999_999]
categorias = ["Até R$ 8.675", "de R$ 8.676 até R$ 25.000", "de R$ 25.001 até R$ 47.000", "Mais de R$ 47.000"]
df_original["FAIXA_VL_TOTAL_PC_PAGAS"] = pd.cut(df_original["VL_TOTAL_PC_PAGAS"], bins=faixas, labels=categorias)
df_original["FAIXA_VL_TOTAL_PC_PAGAS"] = df_original["FAIXA_VL_TOTAL_PC_PAGAS"].cat.set_categories(categorias, ordered = True)
print(f"É ordenada? : {df_original["FAIXA_VL_TOTAL_PC_PAGAS"].cat.ordered}")
print(df_original["FAIXA_VL_TOTAL_PC_PAGAS"].unique())


faixas = [-100, 210_000, 290_000, 400_000, 9_999_999]
categorias = ["Até R$ 210.000", "de R$ 210.000 até R$ 290.000", "de R$ 290.001 até R$ 400.000", "Mais de R$ 400.000"]
df_original["FAIXA_VALOR_FINANCIAMENTO"] = pd.cut(df_original["VL_FINANCIAMENTO"], bins=faixas, labels=categorias)
df_original["FAIXA_VALOR_FINANCIAMENTO"] = df_original["FAIXA_VALOR_FINANCIAMENTO"].cat.set_categories(categorias, ordered = True)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_VALOR_FINANCIAMENTO"].cat.ordered}")
print(df_original["FAIXA_VALOR_FINANCIAMENTO"].unique())

faixas = [-100, 2_500, 3_500, 5_000, 999999]
categorias = ["Até R$ 2.500", "de R$ 2.501 até R$ 3.500", "de R$ 3.501 até R$ 5.000", "Mais de R$ 5.000"]
df_original["FAIXA_VALOR_PARCELA"] = pd.cut(df_original["VL_PARCELA"], bins=faixas, labels=categorias)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_VALOR_PARCELA"].cat.ordered}")
print(df_original["FAIXA_VALOR_PARCELA"].unique())

faixas = [-100, 18, 25, 35, 45, 55, 65, 100]
categorias = ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]
df_original["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"] = pd.cut(df_original["IDADE_DATA_ASSINATURA_CONTRATO"], bins=faixas, labels=categorias)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"].cat.ordered}")
print(df_original["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"].unique())

É ordenada? : True
['Mais de R$ 47.000', 'de R$ 25.001 até R$ 47.000', 'de R$ 8.676 até R$ 25.000', 'Até R$ 8.675']
Categories (4, object): ['Até R$ 8.675' < 'de R$ 8.676 até R$ 25.000' < 'de R$ 25.001 até R$ 47.000' < 'Mais de R$ 47.000']


É ordenada? : True
['Até R$ 210.000', 'de R$ 290.001 até R$ 400.000', 'Mais de R$ 400.000', 'de R$ 210.000 até R$ 290.000']
Categories (4, object): ['Até R$ 210.000' < 'de R$ 210.000 até R$ 290.000' < 'de R$ 290.001 até R$ 400.000' < 'Mais de R$ 400.000']


É ordenada? : True
['Até R$ 2.500', 'de R$ 2.501 até R$ 3.500', 'de R$ 3.501 até R$ 5.000', 'Mais de R$ 5.000']
Categories (4, object): ['Até R$ 2.500' < 'de R$ 2.501 até R$ 3.500' < 'de R$ 3.501 até R$ 5.000' < 'Mais de R$ 5.000']


É ordenada? : True
['65+', '25-34', '55-64', '35-44', '45-54', '18-24', '0-17', NaN]
Categories (7, object): ['0-17' < '18-24' < '25-34' < '35-44' < '45-54' < '55-64' < '65+']


- Criamos faixas de valores em algumas colunas e as ordenamos para diminuirmos a cardinalidade delas
---

In [7835]:
#verificando os intervalos
print(df_original.groupby(["FAIXA_VL_TOTAL_PC_PAGAS"]).size())
print("\n")
print(df_original.groupby(["FAIXA_VALOR_FINANCIAMENTO"]).size())
print("\n")
print(df_original.groupby(["FAIXA_VALOR_PARCELA"]).size())
print("\n")
print(df_original.groupby(["FAIXA_IDADE_DATA_ASSINATURA_CONTRATO"]).size())

FAIXA_VL_TOTAL_PC_PAGAS
Até R$ 8.675                  2603
de R$ 8.676 até R$ 25.000     2702
de R$ 25.001 até R$ 47.000    2512
Mais de R$ 47.000             2593
dtype: int64


FAIXA_VALOR_FINANCIAMENTO
Até R$ 210.000                  3411
de R$ 210.000 até R$ 290.000    2350
de R$ 290.001 até R$ 400.000    2276
Mais de R$ 400.000              2373
dtype: int64


FAIXA_VALOR_PARCELA
Até R$ 2.500                3233
de R$ 2.501 até R$ 3.500    2722
de R$ 3.501 até R$ 5.000    2603
Mais de R$ 5.000            1852
dtype: int64


FAIXA_IDADE_DATA_ASSINATURA_CONTRATO
0-17       43
18-24     731
25-34    3288
35-44    3127
45-54    1761
55-64    1025
65+       434
dtype: int64


In [7836]:
#excluindo as colunas antigas
colunas = ["CIDADE_CLIENTE", "ESTADO_CLIENTE", "RENDA_MENSAL_CLIENTE", "FAIXA_IDADE_DATA_ASSINATURA_CONTRATO", "TAXA_AO_ANO", "PZ_FINANCIAMENTO", "QT_PC_ATRASO", 
           "QT_DIAS_PRIM_PC_ATRASO", "QT_TOTAL_PC_PAGAS", "QT_PC_PAGA_EM_DIA", "QT_DIAS_MIN_ATRASO", "QT_DIAS_MEDIA_ATRASO", "QT_DIAS_MAX_ATRASO", 
           "FAIXA_VL_TOTAL_PC_PAGAS", "FAIXA_VALOR_FINANCIAMENTO", "FAIXA_VALOR_PARCELA", "INADIMPLENTE_COBRANCA"]

df_tratado = pd.DataFrame(df_original, columns = colunas)

In [7837]:
#olhando o dataset para verificarmos a exclusao das colunas antigas e criacao das novas colunas
df_tratado.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CIDADE_CLIENTE                        10410 non-null  category
 1   ESTADO_CLIENTE                        10410 non-null  category
 2   RENDA_MENSAL_CLIENTE                  10410 non-null  float32 
 3   FAIXA_IDADE_DATA_ASSINATURA_CONTRATO  10409 non-null  category
 4   TAXA_AO_ANO                           10410 non-null  float32 
 5   PZ_FINANCIAMENTO                      10410 non-null  int16   
 6   QT_PC_ATRASO                          10410 non-null  int16   
 7   QT_DIAS_PRIM_PC_ATRASO                10410 non-null  int16   
 8   QT_TOTAL_PC_PAGAS                     10410 non-null  int16   
 9   QT_PC_PAGA_EM_DIA                     10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO                    10410 non-null  int16   
 11  QT_DIAS

In [7838]:
#visualizando os dados
df_tratado.sample(10)

,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
1801,GOIANAPOLIS,GO,5800.00,35-44,18.00,180,90,2912,12,11,45,45,45,Até R$ 8.675,Até R$ 210.000,Até R$ 2.500,SIM
1146,LUZIANIA,GO,7800.00,55-64,18.00,72,0,0,49,42,1,12,81,Mais de R$ 47.000,Mais de R$ 400.000,Mais de R$ 5.000,SIM
8869,BRASILIA,DF,7800.00,25-34,18.00,240,100,3037,3,3,0,0,0,Até R$ 8.675,Mais de R$ 400.000,Até R$ 2.500,SIM
9945,SAO PAULO,SP,1800.00,25-34,20.00,100,0,0,35,35,0,0,0,de R$ 25.001 até R$ 47.000,Mais de R$ 400.000,Mais de R$ 5.000,NAO
9586,BRASILIA,DF,7800.00,45-54,15.00,72,0,0,66,66,0,0,0,de R$ 25.001 até R$ 47.000,Até R$ 210.000,Até R$ 2.500,SIM
656,SAO PAULO,SP,9800.00,45-54,19.00,72,0,0,56,44,2,148,1728,de R$ 25.001 até R$ 47.000,de R$ 290.001 até R$ 400.000,de R$ 3.501 até R$ 5.000,NAO
10277,ANAPOLIS,GO,1800.00,25-34,16.50,72,42,1260,1,1,0,0,0,Até R$ 8.675,de R$ 290.001 até R$ 400.000,Mais de R$ 5.000,NAO
1754,GOIANIA,GO,5800.00,55-64,18.00,180,106,3405,5,3,4,5,7,Até R$ 8.675,Até R$ 210.000,Até R$ 2.500,SIM
6733,BRASILIA,DF,7800.00,55-64,16.00,72,5,1627,51,40,1,25,150,de R$ 25.001 até R$ 47.000,Até R$ 210.000,Até R$ 2.500,NAO
1885,BELO HORIZONTE,MG,1800.00,45-54,14.65,72,3,77,10,4,2,24,51,Até R$ 8.675,de R$ 290.001 até R$ 400.000,Mais de R$ 5.000,SIM


## Análise Estatística

In [7839]:
#verificando novamente as métricas para cada coluna
df_tratado.describe() 

,RENDA_MENSAL_CLIENTE,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO
count,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00
mean,5619.98,17.27,114.21,16.69,663.71,35.96,27.68,6.12,28.85,79.16
std,2930.53,2.57,64.31,30.68,1180.94,28.38,25.15,82.93,137.43,334.95
min,1800.00,7.00,48.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,1800.00,16.00,72.00,0.00,0.00,11.00,7.00,0.00,0.00,0.00
50%,7800.00,17.00,72.00,0.00,0.00,33.00,22.00,1.00,4.00,8.00
75%,7800.00,19.00,180.00,22.00,990.00,54.00,42.00,3.00,22.00,62.00
max,9800.00,25.50,240.00,178.00,5655.00,167.00,136.00,5465.00,5480.00,5710.00


In [7840]:
df_tratado.describe(exclude="number")

,CIDADE_CLIENTE,ESTADO_CLIENTE,FAIXA_IDADE_DATA_ASSINATURA_CONTRATO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
count,10410,10410,10409,10410,10410,10410,10410
unique,525,27,7,4,4,4,2
top,BRASILIA,DF,25-34,de R$ 8.676 até R$ 25.000,Até R$ 210.000,Até R$ 2.500,SIM
freq,3734,4237,3288,2702,3411,3233,8035


In [7841]:
#verificando a distribuicao de valores da coluna PZ_Financiamento
print(df_tratado.groupby(["PZ_FINANCIAMENTO"]).size())

PZ_FINANCIAMENTO
48      343
72     6121
100     938
160       2
180     765
194       2
200     558
220     773
240     908
dtype: int64


- A mediana da **Taxa ao Ano** é de **17.27%**.
- Média de **Prazo de Financiamento** é de **114** enquanto a mediana é de **72**. O pulo de 25% dos dados para 50% dos dados não existe, mas de 50% para 75% sai de **72** para **180**. O desvio padrão também é alto, o que indica uma maior dispersão dos dados.
- Algo semelhante occore com as demais colunas do Dataset, onde a maior parte dos valores está concentrada em uma pequena faixa de valores.
- Maior parte dos clientes é de **Brasília**.
- A faixa do **Valor Total de Parcelas Pagas** que mais aparece é de **R$ 8.676 até R$ 25.000**.
- A faixa do **Valor de Financiamento** que mais aparece é de **Até R$ 210.000**.
- A faixa do **Valor da Parcela** que mais aparece é de **Até R$ 2.500**.	
---

## Visualização com Gráficos

In [7842]:
# #grafico de barras mostrando a Distribuição dos Clientes por Renda

# df_count = df_tratado.groupby("RENDA_MENSAL_CLIENTE").size().reset_index(name="Count")

# df_count["RENDA_MENSAL_CLIENTE"] = df_count["RENDA_MENSAL_CLIENTE"]

# total_clientes = df_count["Count"].sum()
# df_count["Porcento"] = (df_count["Count"] / total_clientes) * 100

# #ordenando os dados
# df_count = df_count.sort_values(by="RENDA_MENSAL_CLIENTE", ascending=False)

# fig = px.bar(df_count, x="RENDA_MENSAL_CLIENTE", y="Count", text_auto=True, 
#              template="plotly_dark", 
#              text=df_count["Porcento"],
#              color="Count", color_continuous_scale = "Purples")

# fig.update_traces(dict(marker_line_width=0))
# fig.update_traces(hovertemplate="<b>Renda Mensal: %{x}</b><br>Número de Clientes: %{y}") #Mudando o que aparece quando se coloca o mouse em cima das barras
# fig.update_traces(texttemplate="%{y} Clientes<br>%{text:.2f}%") #mostrando as porcentagens do número total de clientes

# df_count["Cumulativo"] = df_count["Count"].cumsum()
# fig.add_trace(go.Scatter(x=df_count["RENDA_MENSAL_CLIENTE"], #linha amarela que mostra o total de clientes
#                          y=df_count["Cumulativo"], 
#                          mode="lines", 
#                          name="Total Cumulativo", 
#                          line=dict(color="#cc7722", width=2)))


# fig.update_layout(
#     title_text="Distribuição dos Clientes por Renda", #titulo do grafico
#     title_font=dict(size=30, color="#cc7722"), #fonte do titulo do grafico
#     title_x=0.5, #titulo centralizado
#     font=dict(size=14, color="darkgray"), #fonte
#     xaxis_title="Renda Mensal (R$)", #titulo eixo X
#     xaxis_type="category", #tipo do eixo X
#     xaxis_tickformat=",.0f", #formatacao do eixo X
#     xaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo x
#     yaxis_title="Número de Clientes", #titulo do eixo Y
#     yaxis_tickformat=",.0f", #formatacao do eixo Y
#     yaxis_showgrid=True, #mostrando as linhas no eixo Y
#     yaxis_gridwidth=1, #espessura das linhas no eixo Y
#     yaxis_title_font=dict(size=20, color="#cc7722"), #tamanho e cor da fonte no titulo do eixo Y
#     coloraxis_colorbar=dict(x=1.1, #ajusta a posicao do label da barra de cores
#     title="Número de Clientes", #muda o titulo da legenda da barra de cores                 
#     )
# )

# fig.show()

# fig.write_html("html/RENDA_MENSAL_CLIENTE.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/RENDA_MENSAL_CLIENTE.png", width=3440, height=1080 , scale=1) #convertendo para png

- Gráfico mostrando a Distribuição dos Clientes por Renda
---

In [7843]:
# media_salario_idade = df_tratado.groupby("FAIXA_IDADE_DATA_ASSINATURA_CONTRATO")["RENDA_MENSAL_CLIENTE"].mean().reset_index() #achando a media de salario por idade

# media_salario = df_tratado["RENDA_MENSAL_CLIENTE"].mean() #achando a media de salario geral

# top_2 = media_salario_idade.nlargest(2, "RENDA_MENSAL_CLIENTE") #achando os 2 maiores valores da media de salario por idade

# media_salario_idade["Cor"] = media_salario_idade["RENDA_MENSAL_CLIENTE"].apply(   # pintando os 2 maiores valores de roxo, e os demais de cinza claro
#     lambda x: "purple" if x in top_2["RENDA_MENSAL_CLIENTE"].values else "darkgray"
# )

# fig = px.bar(media_salario_idade, x="FAIXA_IDADE_DATA_ASSINATURA_CONTRATO", y="RENDA_MENSAL_CLIENTE", text_auto=True, 
#              color="Cor", color_discrete_sequence=["purple", "darkgray"] ,
#              template="plotly_dark",
#              labels={
#                  "RENDA_MENSAL_CLIENTE": "Renda Mensal Média (R$)",
#                  "FAIXA_IDADE_DATA_ASSINATURA_CONTRATO": "Faixa Etária"
#              },
#              )

# fig.add_shape(type="line", #linha para a media de salario geral
#               x0=-0.5,  
#               y0=media_salario, 
#               x1=len(media_salario_idade) - 0.5,
#               y1=media_salario,
#               line=dict(color="#cc7722", width=3, dash="dash"))


# fig.add_annotation(    #anotacao para a media de salario geral
#     x=len(media_salario_idade) - 1,
#     y=media_salario,
#     text=f"Media: R$ {media_salario:.2f}",
#     showarrow=True,
#     arrowhead=1,
#     arrowcolor="#cc7722",
#     ax=50,
#     ay=60,
#     font=dict(size=16, color="#cc7722")
# )

# fig.update_layout(
#     title_text="Renda Mensal Média por Faixa Etária",
#     title_font=dict(size=30, color="#cc7722"),
#     title_x=0.5,
#     font=dict(size=14, color="darkgray"),
#     xaxis_title="Faixa Etária (Anos)",
#     xaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis_title="Renda Mensal Média (R$)",
#     yaxis_title_font=dict(size=20, color="#cc7722"),
#     yaxis_tickprefix="R$",
# )

# fig.update_traces(textposition="outside")

# fig.for_each_trace(lambda t: t.update(name="2 Maiores Valores" if "purple" in t.marker.color else "Menores Valores")) #mudando a legenda do lado direito

# fig.show()

# fig.write_html("html/RENDA_MENSAL_CLIENTE_FAIXA_ETARIA.html", full_html=False, include_plotlyjs="cdn") #convertendo para html
# pio.write_image(fig, "imagens/RENDA_MENSAL_CLIENTE_FAIXA_ETARIA.png", width=3440, height=1080 , scale=1) #convertendo para png

- A maior média de Renda mensal está entre 0-24 anos
---

In [1]:
# Mapa do Estado do Cliente
# Parcelas pagas em dia x atraso
# Vl_Parcela por Estado
# VL_Parcela por Idade
# VL_Financiamento por Idade
# VL_Financiamento por Estado
# Taxa ao ano por estado
# Taxa ao ano por idade
# QT_PC_ATRASO por estado
# QT_PC_ATRASO por idade
# Line chart showing the relationship between the number of days in delay (QT_DIAS_MAX_ATRASO, QT_DIAS_MIN_ATRASO) and the number of installments in arrears (QT_PC_ATRASO).
#Scatter plot or line chart: Compare TAXA_AO_ANO (annual interest rate) with PZ_FINANCIAMENTO (financing term in months). This can help you understand if longer-term financing tends to have higher or lower interest rates.
#Scatter plot: Compare RENDA_MENSAL_CLIENTE (monthly income) with VALOR_PARCELA (installment value). This could help identify if higher-income customers tend to have higher-value financing.
#Bar chart: Visualize the relationship between QT_PC_ATRASO (number of installments in arrears) and QT_TOTAL_PC_PAGAS (total number of installments paid). This could show trends in how partial arrears relate to overall repayment behavior.
#Scatter plot: Plot VALOR_PARCELA (installment value) against VALOR_FINANCIAMENTO (financing amount). This will help visualize the relationship between the total loan size and how it's structured into installments.
#ine chart or scatter plot: Explore the relationship between IDADE_DATA_ASSINATURA_CONTRATO (client’s age at contract signing) and PZ_FINANCIAMENTO (financing term). You can see if younger or older clients tend to take longer or shorter financing periods.
#Scatter plot: Compare QT_DIAS_MAX_ATRASO (maximum delay in days) with QT_DIAS_MIN_ATRASO (minimum delay in days) for each contract. This might help in identifying variability in payment delays.

## ANALISE EXPLORATORIA MAIS APROFUNDADA COM GRAFICOS DE COLUNAS CATEGORICAS 

In [ ]:
#histogramas
#Bar chart to compare the default rate (INADIMPLENTE_COBRANCA) across different financing types (TIPO_FINANCIAMENTO).
#Heatmap showing the correlation between variables like income (RENDA_MENSAL_CLIENTE), interest rate (TAXA_AO_ANO), delays (QT_DIAS_MEDIA_ATRASO), and the target (INADIMPLENTE_COBRANCA).
#Box plot showing the distribution of monthly income (RENDA_MENSAL_CLIENTE) for clients who defaulted vs. those who didn't.
#Scatter plot or histogram to explore how the client’s age (IDADE_DATA_ASSINATURA_CONTRATO) correlates with the likelihood of default.
#Choropleth map or bar chart showing default rates by state (ESTADO_CLIENTE).
#A pie chart showing the proportion of installments paid on time (QT_PC_PAGAS_EM_DIA) for clients who are in default and not in default.

## ANALISE EXPLORATORIA MAIS APROFUNDADA COM GRAFICOS DE COLUNAS NUMERICAS

In [ ]:
#histograma
#mapa de correlacao

## ONEHOTENCODING

## SEPARAR A VARIAVEL TARGET DAS DEMAIS

## BALANCEAMENTO DA VARIAVEL TARGET

## PADRONIZACAO,NORMALIZACAO,ROBUSTSCALER DOS DADOS

## Conclusão
O modelo de previsão de inadimplência alcançou uma precisão de X%, com uma boa separação entre clientes inadimplentes e não inadimplentes. 

As variáveis mais importantes foram:
- **A**
- **A**


https://dash.plotly.com/tutorial?utm_medium=graphing_libraries&utm_content=python_footer ?
